In [1]:
import dill
import pandas as pd
from tqdm import tqdm
import sys
import pyLDAvis.sklearn
import matplotlib.pyplot

import warnings
warnings.filterwarnings('ignore')

sys.path.append('../scripts')

import twitter
import text_analysis

#Look at topics related to #yanggang, #basicincome, and #ai
basicincome_tweets = dill.load(open('../scrapped_data/twitter-data/basicincome_tweets.pkd', 'rb'))
universalbasicincome_tweets = dill.load(open('../scrapped_data/twitter-data/universalbasicincome_tweets.pkd', 'rb'))
yanggang_tweets = dill.load(open('../scrapped_data/twitter-data/yang_gang_tweets', 'rb'))
ai_tweets = dill.load(open('../scrapped_data/twitter-data/ai_tweets.pkd', 'rb'))

# Look at topics related to Trump, Fake News, and Political Correctness
trump_tweets = dill.load(open('../scrapped_data/twitter-data/trump_tweets.pkd', 'rb'))
fakenews_tweets = dill.load(open('../scrapped_data/twitter-data/fakenews_tweets.pkd', 'rb'))
politicalcorrectness_tweets = dill.load(open('../scrapped_data/twitter-data/politicalcorrectness_tweets.pkd', 'rb'))

# Bernie Tweets
berniebros_tweets = dill.load(open('../scrapped_data/twitter-data/berniebros_tweets.pkd', 'rb'))

# Some Initial EDA

So let us do some initial exploratory data analysis. First, let us check what our common hashtags, and the platform they are coming from.

## Twitter Popularity
Now, let us do a plot of our contestents Twitter activity. I will be measuring two quantitents

1. Retweet count,
2. favorite count.

This will give a proxy for each one's popularity.

### Andrew Yang

In [ ]:
andrew_yang_tweets = dill.load(open('../scrapped_data/twitter-data/yang-tweets.pkd', 'rb'))

In [ ]:
df = pd.DataFrame({'date': [tweet.created_at for tweet in andrew_yang_tweets],
                   'retweets': [tweet.retweet_count for tweet in andrew_yang_tweets],
                   'favorite_count': [tweet.favorite_count for tweet in andrew_yang_tweets]})

In [ ]:
df.plot(x='date', subplots=True, figsize=(16, 12))

### Bernie Sanders

In [ ]:
bernie_sanders_tweets = dill.load(open('../scrapped_data/twitter-data/sanders-tweets.pkd', 'rb'))

In [ ]:
df = pd.DataFrame({'date': [tweet.created_at for tweet in bernie_sanders_tweets],
                   'retweets': [tweet.retweet_count for tweet in bernie_sanders_tweets],
                   'favorite_count': [tweet.favorite_count for tweet in bernie_sanders_tweets]})

In [ ]:
df.plot(x='date', subplots=True, figsize=(16, 12))

### Donald Trump

In [ ]:
donald_trump_tweets = dill.load(open('../scrapped_data/twitter-data/trump_personal_tweets.pkd', 'rb'))

In [ ]:
df = pd.DataFrame({'date': [tweet.created_at for tweet in donald_trump_tweets],
                   'retweets': [tweet.retweet_count for tweet in donald_trump_tweets],
                   'favorite_count': [tweet.favorite_count for tweet in donald_trump_tweets]})

In [ ]:
df.plot(x='date', subplots=True, figsize=(16, 12))

## Word Clouds

In [ ]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt

wordcloud = WordCloud(max_font_size=50, max_words=100).generate(' '.join([tweet.full_text for tweet in donald_trump_tweets]))

plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
wordcloud = WordCloud(max_font_size=50, max_words=100).generate(' '.join([tweet.full_text for tweet in andrew_yang_tweets]))

plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
wordcloud = WordCloud(max_font_size=50, max_words=100).generate(' '.join([tweet.full_text for tweet in bernie_sanders_tweets]))

plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

# Topic Analysis

Next, I gathered Twitter data related to several popular hashtags. The hashtags I chose as relavent are mostly pertinent to issues each campaigner brings up. They are as follows.

1. Andrew Yang
    * #universalbasicincome,
    * #yanggang
    

2. Bernie Sanders
    * #berniebros
    

3. Donald Trump
    * #fakenews,
    * #politicalcorrectness
    * #trump
    
I used [ritetag](https://ritetag.com) to determine some of the hashtags I should be paying attention to. Note at the moment that issues Bernie Sanders is concerned with do not have nice simple hashtags that pop up. This makes EDA somewhat difficult in this case.

First, let's do a basic plot of what kind of people these tweets are coming from.

In [ ]:
sources = ['Twitter for Android', 'Twitter for iPhone', 'Twitter for iPad', 'Twitter Web Client']

basic = pd.DataFrame({'source': ['Andoid', 'iPhone', 'iPad', 'Web Client'],
                      'basicincome': [[tweet.source for tweet in basicincome_tweets].count(s) for s in sources],
                      'yanggang': [[tweet.source for tweet in yanggang_tweets].count(s) for s in sources],
                      'trump': [[tweet.source for tweet in trump_tweets].count(s) for s in sources],
                      'fakenews': [[tweet.source for tweet in fakenews_tweets].count(s) for s in sources],
                      'politicalcorrectness': [[tweet.source for tweet in politicalcorrectness_tweets].count(s) for s in sources]})

In [ ]:
import matplotlib.pyplot

ax = basic.plot(kind='bar')
ax.set_xticklabels(['Andoid', 'iPhone', 'iPad', 'Web Client'])

There isn't any probably any series connection. We could do a $\chi^2$-test to confirm this. The one exception would be is that "Trump" gathers a lot of iPhone users. This could be because Donald Trump, being known as a iPhone user, tweets a lot. Or it could be something inherent about iPhone users. I'm just asking questions here.

Second, why not more love for the iPad?

## Andrew Yang

### Basic Income

In [ ]:
text = twitter.Tweets(basicincome_tweets)

wordcloud = WordCloud(max_font_size=50, max_words=100).generate(' '.join([tweet.full_text for tweet in basicincome_tweets]))

plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [2]:
text = twitter.Tweets(basicincome_tweets)

text.process_tweets_into_df()
text.clean_column()
text.generate_lda_graph()

100%|██████████| 4377/4377 [00:01<00:00, 3782.79it/s]


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
6      310.156311   15.074718       1        1  13.366735
0     -104.232651 -183.014725       2        1  13.009163
5     -211.840317  168.061371       3        1  12.889610
7       59.536453 -303.396454       4        1  11.646883
4       -9.074527  266.324036       5        1  11.358979
3       98.353203  -76.139862       6        1   8.677357
2      262.298889 -205.103943       7        1   7.453854
9      154.599442  145.855499       8        1   7.304623
8     -259.729034  -52.122723       9        1   7.255586
1      -47.959061   39.016216      10        1   7.037212, topic_info=     Category         Freq                  Term        Total  loglift  \
term                                                                     
282   Default   922.000000                 basic   922.000000  30.0000   
1309  Default  1027.000000                income  1027.000000  29.0000   
2692  Default   737.000000                   ubi   737.000000  28.0000   
2847  Default   342.000000                  work   342.000000  27.0000   
2723  Default   467.000000             universal   467.000000  26.0000   
281   Default   315.000000       baseincomequote   315.000000  25.0000   
1427  Default   293.000000                   job   293.000000  24.0000   
1933  Default   412.000000                people   412.000000  23.0000   
2014  Default   283.000000               poverty   283.000000  22.0000   
171   Default   179.000000                   amp   179.000000  21.0000   
1794  Default   290.000000                  need   290.000000  20.0000   
2885  Default   187.000000                  year   187.000000  19.0000   
1805  Default   186.000000                  news   186.000000  18.0000   
179   Default   289.000000            andrewyang   289.000000  17.0000   
1801  Default   369.000000                   new   369.000000  16.0000   
2561  Default   183.000000                   tax   183.000000  15.0000   
257   Default   177.000000            automation   177.000000  14.0000   
1002  Default   139.000000             financial   139.000000  13.0000   
1482  Default   200.000000                  know   200.000000  12.0000   
1188  Default   163.000000                 happy   163.000000  11.0000   
2725  Default   170.000000  universalbasicincome   170.000000  10.0000   
288   Default  2599.000000           basicincome  2599.000000   9.0000   
2502  Default   161.000000             subreddit   161.000000   8.0000   
1007  Default   163.000000               finland   163.000000   7.0000   
2061  Default   200.000000               program   200.000000   6.0000   
2877  Default   156.000000              yanggang   156.000000   5.0000   
1923  Default   181.000000                   pay   181.000000   4.0000   
2873  Default   175.000000              yang2020   175.000000   3.0000   
1748  Default   183.000000                 money   183.000000   2.0000   
2461  Default   135.000000                 start   135.000000   1.0000   
...       ...          ...                   ...          ...      ...   
2438  Topic10    49.869185                 south    50.721759   2.6370   
17    Topic10    47.985096                   170    48.837655   2.6363   
2080  Topic10    46.546845              province    47.399519   2.6358   
2007  Topic10    45.490867                  post    46.343283   2.6354   
1171  Topic10    44.262421              gyeonggi    45.114968   2.6349   
1148  Topic10    40.314662               graphic    41.169015   2.6330   
928   Topic10    35.557799             excellent    36.410762   2.6303   
216   Topic10    35.418374              argument    36.270834   2.6302   
1015  Topic10    32.564179                   fix    33.417052   2.6281   
245   Topic10    32.199745                   auf    33.052579   2.6278   
394   Topic10    31.347243                 build    32.199728   2.6271   
1850 

In this case, I am using this hashtag as a proxy for Universal Basic Income. There are a few things of concern here. Andrew Yang is commonly associated with this movement. Finland also of heavy interest at the moment. This is possibly due to an experimental A/B test Findland ran in 2017-2018 to study the effects of UBI, which [we still don't fully know the data on how that turned out](https://medium.com/basic-income/what-is-there-to-learn-from-finlands-basic-income-experiment-did-it-succeed-or-fail-54b8e5051f60). Also note that there is a lot of concern about the future of artificial intelligence and AI.

### Yang Gang

In [ ]:
text = twitter.Tweets(yanggang_tweets)

wordcloud = WordCloud(max_font_size=50, max_words=100).generate(' '.join([tweet.full_text for tweet in yanggang_tweets]))

plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
df = text.process_tweets_into_df()

tqdm.pandas()
df["processed_text"] = df["full_text"].progress_apply(text_analysis.spacy_tokenizer)

tf, tf_vectorizer, tfidf, tfidf_vectorizer = text_analysis.get_words(df.processed_text.dropna().values)
lda, nmf = text_analysis.get_topics(tf, tfidf, n_components=10)
pyLDAvis.enable_notebook()
dash = pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer, mds='tsne')
dash

A few things of note here. First, Zach Graumann gets mentioned a lot, especially in the first topic. He works as a campaign manager for Andrew Yang. He is also often compared to Trump, which makes sense as if he wins the nomination, he will be Trump's main competitor.

## Bernie Sanders

In [ ]:
text = twitter.Tweets(berniebros_tweets)

wordcloud = WordCloud(max_font_size=50, max_words=100).generate(' '.join([tweet.full_text for tweet in berniebros_tweets]))

plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
df = text.process_tweets_into_df()

tqdm.pandas()
df["processed_text"] = df["full_text"].progress_apply(text_analysis.spacy_tokenizer)

tf, tf_vectorizer, tfidf, tfidf_vectorizer = text_analysis.get_words(df.processed_text.dropna().values)
lda, nmf = text_analysis.get_topics(tf, tfidf, n_components=10)
pyLDAvis.enable_notebook()
dash = pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer, mds='tsne')
dash

## Donald Trump

### Trump

In [ ]:
wordcloud = WordCloud(max_font_size=50, max_words=100).generate(' '.join([tweet.full_text for tweet in trump_tweets]))

plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
text = twitter.Tweets(trump_tweets)
df = text.process_tweets_into_df()

tqdm.pandas()
df["processed_text"] = df["full_text"].progress_apply(text_analysis.spacy_tokenizer)

tf, tf_vectorizer, tfidf, tfidf_vectorizer = text_analysis.get_words(df.processed_text.dropna().values)
lda, nmf = text_analysis.get_topics(tf, tfidf, n_components=10)
pyLDAvis.enable_notebook()
dash = pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer, mds='tsne')
dash

There is a lot to say here. First, these are mostly just concerns. The first topic is on Russia and Twitter, the eigth is about Trump's tax returns, and words like 'impeachment' and 'crime' are associated with multiple topics.

### Fake News

In [ ]:
wordcloud = WordCloud(max_font_size=50, max_words=100).generate(' '.join([tweet.full_text for tweet in fakenews_tweets]))

plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
text = twitter.Tweets(fakenews_tweets)
df = text.process_tweets_into_df()

tqdm.pandas()
df["processed_text"] = df["full_text"].progress_apply(text_analysis.spacy_tokenizer)

tf, tf_vectorizer, tfidf, tfidf_vectorizer = text_analysis.get_words(df.processed_text.dropna().values)
lda, nmf = text_analysis.get_topics(tf, tfidf, n_components=10)
pyLDAvis.enable_notebook()
dash = pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer, mds='tsne')
dash

### Political Correctness

In [ ]:
wordcloud = WordCloud(max_font_size=50, max_words=100).generate(' '.join([tweet.full_text for tweet in politicalcorrectness_tweets]))

plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
text = twitter.Tweets(politicalcorrectness_tweets)
df = text.process_tweets_into_df()

tqdm.pandas()
df["processed_text"] = df["full_text"].progress_apply(text_analysis.spacy_tokenizer)

tf, tf_vectorizer, tfidf, tfidf_vectorizer = text_analysis.get_words(df.processed_text.dropna().values)
lda, nmf = text_analysis.get_topics(tf, tfidf, n_components=10)
pyLDAvis.enable_notebook()
dash = pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer, mds='tsne')
dash